# Notebook for inspecting, regularizing, and cleaning the database of metadata


In [ ]:
# Cell 1: imports and path
import sqlite3
import pandas as pd

# If your notebook is in papers_dashboard/, this path is correct:
DB_PATH = "asr_papers.db"


In [ ]:
# Cell 2: sanity-check that we can connect without error
try:
    with sqlite3.connect(DB_PATH) as conn:
        print(f"✅ Successfully opened {DB_PATH}")
except Exception as e:
    print("❌ Failed to open database:", e)


In [ ]:
# Cell 3: Count how many rows (papers) we have
with sqlite3.connect(DB_PATH) as conn:
    df_total = pd.read_sql("SELECT COUNT(*) AS total_papers FROM papers;", conn)

print("Total papers loaded:", df_total["total_papers"].iloc[0])


In [ ]:
# Cell 4a: Show table_info (cid, name, type, notnull, dflt_value, pk)
with sqlite3.connect(DB_PATH) as conn:
    df_schema = pd.read_sql("PRAGMA table_info(papers);", conn)

df_schema


In [ ]:
# Cell 5: Count NULL or blank entries for every column
with sqlite3.connect(DB_PATH) as conn:
    df_schema = pd.read_sql("PRAGMA table_info(papers);", conn)

print("Missing/NULL counts per column:\n")
for col in df_schema["name"]:
    col_type = df_schema[df_schema["name"] == col]["type"].iloc[0].upper()
    if col_type == "INTEGER":
        query = f"SELECT COUNT(*) AS missing_{col} FROM papers WHERE {col} IS NULL;"
    else:
        query = f"SELECT COUNT(*) AS missing_{col} FROM papers WHERE {col} IS NULL OR trim({col}) = '';"
    df_missing = pd.read_sql(query, conn)
    missing_count = df_missing.iloc[0, 0]
    print(f"  {col:<18} → {missing_count} missing")


In [ ]:
# Cell 6: Distinct values for genre, journal, and source_quality
with sqlite3.connect(DB_PATH) as conn:
    for colname in ["genre", "journal", "source_quality", "publisher"]:
        df_distinct = pd.read_sql(
            f"SELECT DISTINCT {colname} AS val FROM papers ORDER BY {colname};", conn
        )
        values = df_distinct["val"].fillna("<NULL>").tolist()
        print(f"\nDistinct values in '{colname}':")
        for v in values:
            print("   ", repr(v))


In [ ]:
# Cell 7: Preview first 5 rows of the table
with sqlite3.connect(DB_PATH) as conn:
    df_preview = pd.read_sql("SELECT * FROM papers LIMIT 5;", conn)

df_preview
